In [1]:
#%pdb

In [2]:
import math
from sortedcontainers import SortedDict
import random
import numpy 
import matplotlib.pyplot as plt
from scipy.stats import truncnorm

In [3]:
# Code for generating positions, distances, travel times and preferences
def generate_positions(males, x_dim, y_dim):
    Xs = numpy.random.rand(males) * x_dim
    Ys = numpy.random.rand(males) * y_dim
    return [Xs,Ys]

def compute_distances_travel_times(males, positions, bird_speed):
    male_dist = numpy.zeros((males, males))
    travel_times = numpy.zeros((males, males))
    for i in range(males):
        for j in range(i + 1, males):
            dist = math.sqrt((positions[0][j] - positions[0][i]) ** 2 + (positions[1][j] - positions[1][i]) ** 2)
            travel = dist / bird_speed
            male_dist[j][i] = dist
            male_dist[i][j] = dist
            travel_times[j][i] = travel
            travel_times[i][j] = travel
    return (male_dist, travel_times)

def compute_visit_preferences(males, distances, lambda_dist):
    # compute exponential of each coefficient
    visit_preferences = numpy.exp(-lambda_dist * distances)
    # remove the identity matrix (exp(0) = 1)
    visit_preferences = visit_preferences - numpy.eye(males)
    # make rows sum to one
    visit_preferences = (visit_preferences.transpose() / numpy.sum(visit_preferences, 1)).transpose()
    return visit_preferences


In [4]:
# functions to generate tickets and manage timeline
def generate_ticket(start_time, end_time, length_activity, owner, action, target):
    global timeline
    ticket = {"start_time": start_time,
              "end_time": end_time,
              "length_activity": length_activity,
              "owner": owner,
              "action": action,
              "target": target
             }
    # now add to timeline
    timeline[(ticket["end_time"], ticket["owner"])] = ticket


In [5]:
# ACTION FUNCTIONS
# Each action generates a ticket, and updates the state of the owner (and possibly the target)

def draw_foraging_time(start_time):
    time_between = FG_tau_std * truncnorm.rvs(FG_tau_norm_range[0], FG_tau_norm_range[1]) + FG_tau_mean
    return start_time + time_between

def action_forage(bird_id, current_time):
    global birds
    # generate the time it takes to forage
    time_spent_foraging = numpy.random.gamma(FG_k, FG_theta)/FG_divisor
    time_action_ends = current_time + time_spent_foraging
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_foraging,
                   owner = bird_id,
                   action = "foraging",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "foraging"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends
    # update the time to next foraging: start counting when foraging ended
    birds[bird_id]["next_foraging_time"] = draw_foraging_time(time_action_ends)

def action_stay_at_bower(bird_id, current_time):
    global birds
    # generate the length of the stay
    time_spent_at_bower = RBSB_tau_std * truncnorm.rvs(RBSB_tau_norm_range[0], RBSB_tau_norm_range[1]) + RBSB_tau_mean
    time_action_ends = current_time + time_spent_at_bower
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_at_bower,
                   owner = bird_id,
                   action = "staying at bower",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "staying at bower"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends

def action_travel_to_maraud(bird_id, current_time):
    global birds
    # choose who to maraud
    tmp = numpy.random.rand()
    target = numpy.argwhere(birds[bird_id]["travel_preferences"] > tmp)[0][0] 
    time_to_travel = birds[bird_id]["travel_times"][target]
    time_action_ends = current_time + time_to_travel
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_to_travel,
                   owner = bird_id,
                   action = "travel to maraud",
                   target = target)
    # update the bird:
    birds[bird_id]["current_state"] = "travel to maraud"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends

def action_repair_bower(bird_id, current_time):
    global birds
    # generate the length of the repair bout
    time_spent_repairing_bower = RBSB_tau_std * truncnorm.rvs(RBSB_tau_norm_range[0], RBSB_tau_norm_range[1]) + RBSB_tau_mean
    time_action_ends = current_time + time_spent_repairing_bower
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_repairing_bower,
                   owner = bird_id,
                   action = "repairing bower",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "repairing bower"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends
    # note: already accounts for the improvements
    birds[bird_id]["bower_state"] = birds[bird_id]["bower_state"] + time_spent_repairing_bower
    # cannot make it better than 0
    if birds[bird_id]["bower_state"] > 0.0:
        birds[bird_id]["bower_state"] = 0.0

def action_maraud(marauder_id, marauder_target, current_time):
    global birds
    # note: HARD CODED PARAMS!
    time_spent_marauding = 0.1 
    # note: HARD CODED PARAMS!
    damage_to_bower = 6.0 
    time_action_ends = current_time + time_spent_marauding
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_marauding,
                   owner = marauder_id,
                   action = "marauding",
                   target = marauder_target)
    # update marauder
    birds[marauder_id]["current_state"] = "marauding"
    birds[marauder_id]["action_starts"] = current_time
    birds[marauder_id]["action_ends"] = time_action_ends
    # update target
    birds[marauder_target]["bower_state"] = birds[marauder_target]["bower_state"] - damage_to_bower
    

def action_travel_from_maraud(marauder_id, marauder_target, current_time):
    global birds
    time_from_travel = birds[marauder_id]["travel_times"][marauder_target]
    time_action_ends = current_time + time_from_travel
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_from_travel,
                   owner = marauder_id,
                   action = "travel from maraud",
                   target = marauder_target)
    # update the bird:
    birds[marauder_id]["current_state"] = "travel from maraud"
    birds[marauder_id]["action_starts"] = current_time
    birds[marauder_id]["action_ends"] = time_action_ends
    
def action_mating_attempt(female_id, current_time):
    global birds
    global female_birds
    female_index = int(female_id[1:]) #convert from ID to index; i.e. "F0" -> 0
    last_location=female_birds[female_index]["already_visited"][-1:][0] #index of most recently visited male
    if len(female_birds[female_index]["already_visited"]) == female_birds[female_index]["max_per_day"]:
        female_birds[female_index]["already_visited"] = []
        tmp = numpy.random.rand()
        target = numpy.argwhere(birds[last_location]["travel_preferences"] > tmp)[0][0]
        time_to_travel = birds[last_location]["travel_times"][target]
        time_action_ends = current_time + time_to_travel
        generate_ticket(start_time = time_action_ends + 12, #HARD CODE
                        end_time = time_action_ends + 12,
                        length_activity = 0,
                        owner = female_id,
                        action = "mating attempt",
                        target = target)
    else:        
        p = list(visit_preferences[last_location]) #list of preferences by index (has probability -- not cumulative)
        for i in female_birds[female_index]["already_visited"]:
            p[i] = 0.0
        p = numpy.cumsum(p)
        scale_rand = p[-1:]
        tmp = numpy.random.rand() * scale_rand
        target = numpy.argwhere(p > tmp)[0][0]
        time_to_travel = birds[last_location]["travel_times"][target]
        time_action_ends = current_time + time_to_travel
        generate_ticket(start_time = time_action_ends, #HARD CODE
                        end_time = time_action_ends,
                        length_activity = 0,
                        owner = female_id,
                        action = "mating attempt",
                        target = target)
        

In [6]:
# this function should be called every time the bird 
# 1) is back to the bower (from foraging, marauding), 
# 2) has finished repairing the bower, 
# 3) or has finished a stint at staying at bower
def choose_action(bird, current_time):
    global t_max
    # stop generating actions at t_max
    if current_time < t_max:
        # if it's time to eat
        if current_time > bird["next_foraging_time"]:
            action_forage(bird["id"], current_time)
        # if the bower needs repair
        elif bird["bower_state"] < 0.0:
            # go repair
            action_repair_bower(bird["id"], current_time)
        # check if it wants to maraud
        elif numpy.random.rand() < bird["probability_maraud"]:
            # go maraud
            action_travel_to_maraud(bird["id"], current_time)
        else:
            # stay at bower
            action_stay_at_bower(bird["id"], current_time)
       

In [7]:
def initialize_bird(bird_id, bird_strategy, bird_xy, bird_preferences, bird_travel_times):
    # initialize dictionary
    bird = {"id": bird_id,
            "current_state": "none",
            "action_starts": 0.0,
            "action_ends": -1.0,
            "probability_maraud": bird_strategy,
            "bower_state": 0.0,
            "successful_mating": 0,
            "next_foraging_time": draw_foraging_time(0.0),
            "travel_preferences": numpy.cumsum(bird_preferences), # note: store cumulative probability for faster choice
            "travel_times": bird_travel_times, 
            "position": bird_xy
            }
    return(bird)

In [8]:
def initialize_female(female_id):
    #initialize dictionary
    female_bird = {"id": female_id,
             "already_visited": [numpy.random.choice(list(range(males)))], #numpy.random.choice(list(range(males)))
             "max_per_day": min(males-1, 6),
             "wait_period": 12,
            #HARD CODED PARAMS
            }
    return(female_bird)

In [9]:
# this is the most important function!
def read_ticket(tic):
    global birds
    global t_max
    if tic["action"] in ("foraging", "staying at bower", "repairing bower", "travel from maraud"):
        # I am back at the bower, choose new action
        choose_action(birds[tic["owner"]], tic["end_time"])
    elif tic["action"] == "travel to maraud":
        # check whether the target is at home
        go_maraud = True
        if birds[tic["target"]]["current_state"] in ("staying at bower", "repairing bower"):
            go_maraud = False
        if birds[tic["target"]]["bower_state"] < 0.0:
            go_maraud = False
        if go_maraud: # maraud
            action_maraud(tic["owner"], tic["target"], tic["end_time"])
        else: # go back
            action_travel_from_maraud(tic["owner"], tic["target"], tic["end_time"])
    elif tic["action"] == "marauding":
        # travel back
        action_travel_from_maraud(tic["owner"], tic["target"], tic["end_time"])
    elif tic["action"] == "mating attempt":
        if birds[tic["target"]]["current_state"] == "staying at bower": #if male is at bower and it is intact
            birds[tic["target"]]["successful_mating"] += 1 #successfully mate and stop generating tickets
        else:  
            female_birds[int(tic["owner"][1:])]["already_visited"].append(tic["target"]) #update the female's already_visited list
            if tic['end_time'] < t_max:
                action_mating_attempt(tic["owner"],tic["end_time"]) #generate a new ticket
    else:
        1 / 0 # something went horribly wrong
    

In [10]:
# Global Variables

# TIMELINE
timeline = SortedDict()
t_max = 12*10 #12 * 30 # time when simulation ends

# MALES
males = 100 # number of male birds

# FEMALES
female_birds=[]
F_per_M = 3 #The number of sexualy mature females per sexually mature male
females = males*F_per_M # number of female birds
female_visit_param= [0, t_max/2]

# POSITIONS AND TRAVEL TIME
x_dim, y_dim = 400, 500 # dimensions of environment
bird_speed = 12 * 3600 # m/hr (12 m/s)
# now choose lambda_dist, controlling the probability of traveling to a neighbor
# the probability of choosing a neighbor at distance x is proportional to exp(-\lambda x)
# choose lambda such that 99% of the mass is before 800 meters
improb = 0.99
improb_distance = 800
lambda_dist = - math.log(1.0 - improb) / improb_distance

# ACTION DISTRIBUTIONS
# Time of forage
FG_tau_mean, FG_tau_std = .4, .167 #mean and sd of truncated normal distribution rv to find a male's time until next FG
FG_tau_range = [0, 1] #maximum and minimum FG taus
FG_tau_norm_range = [(FG_tau_range[0] - FG_tau_mean) / FG_tau_std, (FG_tau_range[1] - FG_tau_mean) / FG_tau_std] #normalized
# Duration of forage
FG_k=1.5 #the shape of the gamma distribution rv used to generate FG taus
FG_theta=5 #the scale of the gamma distribution rv used to generate FG taus
FG_divisor=60
# Duration of repair bower / stay at bower
RBSB_tau_mean, RBSB_tau_std = .1583, .09755 #mean and sd of truncated normal distribution rv to find duration of repair bower / stay at bower
RBSB_tau_range = [0,.5] #maximum and minimum taus
RBSB_tau_norm_range = [(RBSB_tau_range[0] - RBSB_tau_mean) / RBSB_tau_std, (RBSB_tau_range[1] - RBSB_tau_mean) / RBSB_tau_std] #normalized



# BIRDS
birds = []
strategies = numpy.random.random(males)

# DEBUG
past_events = []
timeline_lengths = []

######## CODE SIMULATION #########
numpy.random.seed(0)

# initialize positions, travel times and preferences
positions = generate_positions(males, x_dim, y_dim)
distances, travel_times = compute_distances_travel_times(males, positions, bird_speed)
visit_preferences = compute_visit_preferences(males, distances, lambda_dist)
for i in range(males):
    birds.append(initialize_bird(i, 
                                 strategies[i], 
                                 (positions[0][i], positions[1][i]), 
                                 visit_preferences[i],
                                 travel_times[i]))
    # choose its first action
    choose_action(birds[-1], 0.0)

#initialize females
for i in range(females): #females
    female_id = "F{:d}".format(i)
    female_birds.append(initialize_female(female_id)) #female IDs start where males end (if there are 10 males, the first female would be 11)
    #choose time for initial mating attempt
    first_time = numpy.random.uniform(female_visit_param[0], female_visit_param[1])
    action_mating_attempt(female_id, first_time)
    
# this is the main loop
while len(timeline) > 0:
    current_ticket = timeline.popitem(0)
    read_ticket(current_ticket[1])
    # for debug: store all the past tickets
    past_events.append(current_ticket)
    # and every so often check the length of the timeline
    if numpy.random.rand() < 0.01:
        timeline_lengths.append(len(timeline))


In [11]:
# DEBUG: check that timeline does not bloat!
print(timeline_lengths)

[400, 399, 399, 399, 399, 398, 398, 398, 398, 398, 398, 398, 398, 398, 398, 398, 398, 398, 397, 397, 397, 397, 397, 397, 397, 397, 397, 397, 396, 396, 396, 396, 396, 396, 396, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 395, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 394, 393, 393, 393, 393, 393, 393, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 392, 391, 391, 391, 391, 391, 391, 391, 391, 391, 391, 391, 391, 391, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 390, 389,

In [12]:
# DEBUG: MAKE SURE THAT TIMELINE MAKES SENSE
for my_bird in range(males):
    print("#################################")
    print("TIMELINE OF BIRD", my_bird)
    print("#################################")
    bird_actions = {}
    for event in past_events:
        tic = event[1]
        if tic["owner"] == my_bird:
            print(round(tic["start_time"], 3), "-", round(tic["end_time"], 3),
                 "->", tic["action"], "[target:", tic["target"], "]")
    print("#################################")
    print("")
    print("") 
    

    

#################################
TIMELINE OF BIRD 0
#################################
0.0 - 0.136 -> staying at bower [target: -1 ]
0.136 - 0.174 -> staying at bower [target: -1 ]
0.174 - 0.478 -> staying at bower [target: -1 ]
0.478 - 0.582 -> foraging [target: -1 ]
0.582 - 0.616 -> staying at bower [target: -1 ]
0.616 - 0.77 -> staying at bower [target: -1 ]
0.77 - 0.973 -> staying at bower [target: -1 ]
0.973 - 1.158 -> staying at bower [target: -1 ]
1.158 - 1.573 -> foraging [target: -1 ]
1.573 - 1.879 -> staying at bower [target: -1 ]
1.879 - 2.223 -> staying at bower [target: -1 ]
2.223 - 2.316 -> foraging [target: -1 ]
2.316 - 2.73 -> staying at bower [target: -1 ]
2.73 - 2.931 -> foraging [target: -1 ]
2.931 - 3.061 -> staying at bower [target: -1 ]
3.061 - 3.301 -> staying at bower [target: -1 ]
3.301 - 3.315 -> staying at bower [target: -1 ]
3.315 - 3.41 -> staying at bower [target: -1 ]
3.41 - 3.418 -> foraging [target: -1 ]
3.418 - 3.421 -> travel to maraud [target: 39 ]
3

25.706 - 25.89 -> repairing bower [target: -1 ]
25.89 - 25.894 -> foraging [target: -1 ]
25.894 - 25.896 -> travel to maraud [target: 23 ]
25.896 - 25.996 -> marauding [target: 23 ]
25.996 - 25.998 -> travel from maraud [target: 23 ]
25.998 - 26.081 -> staying at bower [target: -1 ]
26.081 - 26.088 -> travel to maraud [target: 54 ]
26.088 - 26.095 -> travel from maraud [target: 54 ]
26.095 - 26.228 -> staying at bower [target: -1 ]
26.228 - 26.398 -> staying at bower [target: -1 ]
26.398 - 26.437 -> staying at bower [target: -1 ]
26.437 - 26.482 -> foraging [target: -1 ]
26.482 - 26.485 -> travel to maraud [target: 17 ]
26.485 - 26.487 -> travel from maraud [target: 17 ]
26.487 - 26.836 -> staying at bower [target: -1 ]
26.836 - 27.097 -> staying at bower [target: -1 ]
27.097 - 27.358 -> foraging [target: -1 ]
27.358 - 27.506 -> staying at bower [target: -1 ]
27.506 - 27.733 -> staying at bower [target: -1 ]
27.733 - 28.067 -> foraging [target: -1 ]
28.067 - 28.182 -> staying at bower 

73.168 - 73.287 -> staying at bower [target: -1 ]
73.287 - 73.322 -> foraging [target: -1 ]
73.322 - 73.432 -> staying at bower [target: -1 ]
73.432 - 73.645 -> staying at bower [target: -1 ]
73.645 - 73.935 -> staying at bower [target: -1 ]
73.935 - 74.004 -> foraging [target: -1 ]
74.004 - 74.244 -> staying at bower [target: -1 ]
74.244 - 74.417 -> staying at bower [target: -1 ]
74.417 - 74.48 -> staying at bower [target: -1 ]
74.48 - 74.657 -> foraging [target: -1 ]
74.657 - 74.708 -> repairing bower [target: -1 ]
74.708 - 74.763 -> repairing bower [target: -1 ]
74.763 - 74.856 -> repairing bower [target: -1 ]
74.856 - 75.015 -> repairing bower [target: -1 ]
75.015 - 75.317 -> repairing bower [target: -1 ]
75.317 - 75.337 -> foraging [target: -1 ]
75.337 - 75.43 -> repairing bower [target: -1 ]
75.43 - 75.567 -> repairing bower [target: -1 ]
75.567 - 75.942 -> repairing bower [target: -1 ]
75.942 - 76.149 -> foraging [target: -1 ]
76.149 - 76.378 -> repairing bower [target: -1 ]
76.

100.259 - 100.321 -> foraging [target: -1 ]
100.321 - 100.581 -> staying at bower [target: -1 ]
100.581 - 100.734 -> staying at bower [target: -1 ]
100.734 - 100.935 -> staying at bower [target: -1 ]
100.935 - 101.094 -> foraging [target: -1 ]
101.094 - 101.332 -> staying at bower [target: -1 ]
101.332 - 101.633 -> staying at bower [target: -1 ]
101.633 - 101.779 -> foraging [target: -1 ]
101.779 - 101.949 -> repairing bower [target: -1 ]
101.949 - 102.134 -> foraging [target: -1 ]
102.134 - 102.368 -> repairing bower [target: -1 ]
102.368 - 102.556 -> repairing bower [target: -1 ]
102.556 - 102.714 -> repairing bower [target: -1 ]
102.714 - 102.841 -> repairing bower [target: -1 ]
102.841 - 102.959 -> foraging [target: -1 ]
102.959 - 103.025 -> repairing bower [target: -1 ]
103.025 - 103.236 -> repairing bower [target: -1 ]
103.236 - 103.407 -> repairing bower [target: -1 ]
103.407 - 103.49 -> foraging [target: -1 ]
103.49 - 103.577 -> repairing bower [target: -1 ]
103.577 - 103.796 -

133.6 - 133.854 -> repairing bower [target: -1 ]
133.854 - 133.933 -> repairing bower [target: -1 ]
133.933 - 134.313 -> repairing bower [target: -1 ]
134.313 - 134.36 -> foraging [target: -1 ]
134.36 - 134.596 -> repairing bower [target: -1 ]
134.596 - 134.75 -> foraging [target: -1 ]
134.75 - 134.904 -> repairing bower [target: -1 ]
134.904 - 135.038 -> repairing bower [target: -1 ]
135.038 - 135.305 -> repairing bower [target: -1 ]
135.305 - 135.323 -> foraging [target: -1 ]
135.323 - 135.551 -> repairing bower [target: -1 ]
135.551 - 135.665 -> repairing bower [target: -1 ]
135.665 - 136.182 -> foraging [target: -1 ]
136.182 - 136.317 -> repairing bower [target: -1 ]
136.317 - 136.409 -> repairing bower [target: -1 ]
136.409 - 136.482 -> repairing bower [target: -1 ]
136.482 - 136.577 -> repairing bower [target: -1 ]
136.577 - 136.708 -> repairing bower [target: -1 ]
136.708 - 136.773 -> foraging [target: -1 ]
136.773 - 136.855 -> repairing bower [target: -1 ]
136.855 - 136.955 -> 

...

379.042 - 379.286 -> foraging [target: -1 ]
379.286 - 379.523 -> repairing bower [target: -1 ]
379.523 - 379.635 -> repairing bower [target: -1 ]
379.635 - 379.715 -> foraging [target: -1 ]
379.715 - 379.899 -> repairing bower [target: -1 ]
379.899 - 379.951 -> foraging [target: -1 ]
379.951 - 379.955 -> travel to maraud [target: 60 ]
379.955 - 379.958 -> travel from maraud [target: 60 ]
379.958 - 379.962 -> travel to maraud [target: 60 ]
379.962 - 379.966 -> travel from maraud [target: 60 ]
379.966 - 379.971 -> travel to maraud [target: 95 ]
379.971 - 379.977 -> travel from maraud [target: 95 ]
379.977 - 380.165 -> staying at bower [target: -1 ]
380.165 - 380.171 -> travel to maraud [target: 7 ]
380.171 - 380.177 -> travel from maraud [target: 7 ]
380.177 - 380.179 -> travel to maraud [target: 3 ]
380.179 - 380.18 -> travel from maraud [target: 3 ]
380.18 - 380.182 -> travel to maraud [target: 65 ]
380.182 - 380.183 -> travel from maraud [target: 65 ]
380.183 - 380.192 -> travel to ma

391.662 - 391.663 -> travel from maraud [target: 64 ]
391.663 - 391.668 -> travel to maraud [target: 68 ]
391.668 - 391.673 -> travel from maraud [target: 68 ]
391.673 - 391.936 -> staying at bower [target: -1 ]
391.936 - 392.068 -> foraging [target: -1 ]
392.068 - 392.077 -> travel to maraud [target: 92 ]
392.077 - 392.086 -> travel from maraud [target: 92 ]
392.086 - 392.087 -> travel to maraud [target: 51 ]
392.087 - 392.088 -> travel from maraud [target: 51 ]
392.088 - 392.387 -> staying at bower [target: -1 ]
392.387 - 392.391 -> travel to maraud [target: 85 ]
392.391 - 392.395 -> travel from maraud [target: 85 ]
392.395 - 392.397 -> travel to maraud [target: 84 ]
392.397 - 392.399 -> travel from maraud [target: 84 ]
392.399 - 392.406 -> travel to maraud [target: 67 ]
392.406 - 392.414 -> travel from maraud [target: 67 ]
392.414 - 392.418 -> travel to maraud [target: 30 ]
392.418 - 392.422 -> travel from maraud [target: 30 ]
392.422 - 392.424 -> travel to maraud [target: 47 ]
392.

411.947 - 412.057 -> repairing bower [target: -1 ]
412.057 - 412.392 -> repairing bower [target: -1 ]
412.392 - 412.46 -> foraging [target: -1 ]
412.46 - 412.571 -> repairing bower [target: -1 ]
412.571 - 412.572 -> travel to maraud [target: 22 ]
412.572 - 412.573 -> travel from maraud [target: 22 ]
412.573 - 412.577 -> travel to maraud [target: 18 ]
412.577 - 412.58 -> travel from maraud [target: 18 ]
412.58 - 412.581 -> travel to maraud [target: 22 ]
412.581 - 412.582 -> travel from maraud [target: 22 ]
412.582 - 412.589 -> travel to maraud [target: 67 ]
412.589 - 412.597 -> travel from maraud [target: 67 ]
412.597 - 412.604 -> travel to maraud [target: 70 ]
412.604 - 412.611 -> travel from maraud [target: 70 ]
412.611 - 412.671 -> staying at bower [target: -1 ]
412.671 - 412.679 -> travel to maraud [target: 4 ]
412.679 - 412.687 -> travel from maraud [target: 4 ]
412.687 - 412.693 -> travel to maraud [target: 20 ]
412.693 - 412.699 -> travel from maraud [target: 20 ]
412.699 - 412.7

426.052 - 426.24 -> repairing bower [target: -1 ]
426.24 - 426.257 -> foraging [target: -1 ]
426.257 - 426.347 -> repairing bower [target: -1 ]
426.347 - 426.63 -> repairing bower [target: -1 ]
426.63 - 426.743 -> foraging [target: -1 ]
426.743 - 426.905 -> repairing bower [target: -1 ]
426.905 - 427.078 -> repairing bower [target: -1 ]
427.078 - 427.302 -> repairing bower [target: -1 ]
427.302 - 427.41 -> foraging [target: -1 ]
427.41 - 427.54 -> repairing bower [target: -1 ]
427.54 - 427.608 -> repairing bower [target: -1 ]
427.608 - 427.896 -> repairing bower [target: -1 ]
427.896 - 428.068 -> foraging [target: -1 ]
428.068 - 428.185 -> repairing bower [target: -1 ]
428.185 - 428.459 -> repairing bower [target: -1 ]
428.459 - 428.764 -> repairing bower [target: -1 ]
428.764 - 428.892 -> foraging [target: -1 ]
428.892 - 429.116 -> repairing bower [target: -1 ]
429.116 - 429.133 -> foraging [target: -1 ]
429.133 - 429.252 -> repairing bower [target: -1 ]
429.252 - 429.254 -> repairing

KeyboardInterrupt: 

In [13]:
[]
for i in range(males):
    print("bird: {:d}; matings: {:d}; strategy: {:f}".format(i, birds[i]["successful_mating"], birds[i]["probability_maraud"]))



bird: 0; matings: 2; strategy: 0.175622
bird: 1; matings: 4; strategy: 0.191652
bird: 2; matings: 5; strategy: 0.195916
bird: 3; matings: 0; strategy: 0.996433
bird: 4; matings: 6; strategy: 0.325280
bird: 5; matings: 8; strategy: 0.066511
bird: 6; matings: 2; strategy: 0.486038
bird: 7; matings: 6; strategy: 0.233166
bird: 8; matings: 3; strategy: 0.913270
bird: 9; matings: 1; strategy: 0.897466
bird: 10; matings: 4; strategy: 0.405284
bird: 11; matings: 5; strategy: 0.162894
bird: 12; matings: 3; strategy: 0.708586
bird: 13; matings: 6; strategy: 0.090818
bird: 14; matings: 0; strategy: 0.559854
bird: 15; matings: 0; strategy: 0.679853
bird: 16; matings: 3; strategy: 0.042359
bird: 17; matings: 3; strategy: 0.008862
bird: 18; matings: 1; strategy: 0.771604
bird: 19; matings: 3; strategy: 0.492413
bird: 20; matings: 1; strategy: 0.884221
bird: 21; matings: 6; strategy: 0.112718
bird: 22; matings: 3; strategy: 0.229270
bird: 23; matings: 3; strategy: 0.729793
bird: 24; matings: 3; stra

In [ ]:
# DEBUG: MAKE SURE THAT FEMALE TIMELINE MAKES SENSE
for i in range(females):
    my_bird = "F{:d}".format(i)
    print("#################################")
    print("TIMELINE OF BIRD", my_bird)
    print("#################################")
    bird_actions = {}
    for event in past_events:
        tic = event[1]
        if tic["owner"] == my_bird:
            print(round(tic["start_time"], 3), "-", round(tic["end_time"], 3),
                 "->", tic["action"], "[target:", tic["target"], "]")
    print("#################################")
    print("")
    print("") 